# Abstract

The Fama-French 3 Factors Model allows investors to conduct a Factor Analysis in order to analyse the performance of a fund manager or ETF by dissecting past returns over different factors (Market, Size, Value). In this project, we analyse four ETFs from iShares, which target and are highly exposed to at least one of these factors. Results confirmed the targetted exposure of these products, resulting in low alphas and a good fit of the FF3F model. Two portfolio (Technology and Growth) showed positive alphas which we interpret as an hidden technology factor. The selected ETFs provided a relevant framework to assess the model’s fitness and further research could explore factors identification using models such as Principal Component Analysis (PCA).

# Fama-French 3 Factors Model : Study of 4 ETF

The FF3F model can be used to evaluate a fund performance. Especially, it points out the excess return of an active management over a benchmark (i.e. the risk factors highlight the systematic returns brought by the factors).

The model is based on an econometric multiple linear regression:

$R_i - R_f = \alpha_i + \beta_{MKT} (R_m - R_f) + \beta_{SMB} \, SMB + \beta_{HML} \, HML + \varepsilon_i$

where:
- $R_i$ : fund returns
- $R_f$ : risk-free rate
- $R_m - R_f$ : excess returns from the market *(Market Factor)*
- $SMB$ : Small Minus Big *(Size Factor)*
- $HML$ : High Minus Low *(Value Factor)*
- $\alpha_i$ : Managemer performance *(Alpha)*
- $\varepsilon_i$ : Error term

The Tuck School of Business from Darthmouth publishes frequent updated data related to this model on the Kenneth R. French - Data Library website. They calculate the factors return from the CRSP Value-Weighted Market Index which gathers every US stocks from NYSE, AMEX and NASDAQ.

We are going to use the data made available by this library in order to analyze the risk factors expositions of 4 ETF managed by iShares presented below (Small-Cap, Technology, Growth, Value) and notice eventual Alpha.

We study a span of 10 years from July 2015 to July 2025 (last available data) to be able to add a macro analysis if necessary.

# Packages and data collection

In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

For this Fama-French 3 Factors analysis, we choose 4 ETFs managed by iShares (BlackRock).

Each ETF hold only stocks listed on the U.S. market for a coherent comparison with the factors data used.

Here are the 4 ETFs labelled after their targetted factor:
- **iShares Core S&P Small-Cap ETF**
- **iShares U.S. Technology ETF**
- **iShares S&P 500 Growth ETF**
- **iShares S&P 500 Value ETF**


In [2]:
# iShares Core S&P Small-Cap ETF
small_cap = yf.download('IJR', auto_adjust=False, start="2000-01-01")

# iShares U.S. Technology ETF
tech = yf.download('IYW', auto_adjust=False, start="2000-01-01")

# iShares S&P 500 Growth ETF
growth = yf.download('IVW', auto_adjust=False, start="2000-01-01")

# iShares S&P 500 Value ETF
value = yf.download('IVE', auto_adjust=False, start="2000-01-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Functions and examples

In [3]:
def monthly_returns(asset):
    """
    Get monthly returns from a Series of prices.

    Parameters
    ----------
    asset : pd.DataFrame
        raw data obtained with yf.download('Ticker', auto_adjust = False)
    """
    asset_price = asset[['Adj Close']].droplevel('Ticker', axis=1)
    asset_m = asset_price.resample('ME').last().pct_change().dropna() # monthly returns
    asset_m.index = asset_m.index.to_period("M")
    return asset_m

In [4]:
monthly_returns(small_cap).tail()

Price,Adj Close
Date,
2025-06,0.041002
2025-07,0.009058
2025-08,0.070366
2025-09,0.010477
2025-10,-0.004965


In [5]:
def get_fff():
    """
    Import Fama French 3 Factors data from a CSV donwloanded from the Kenneth R. French library.
    """
    fff_m = pd.read_csv("fff_m_ret.csv", index_col=0)
    fff_m = fff_m.replace(',', '.', regex=True).astype(float) / 100 # returns expressed in % in the library
    fff_m.index = pd.to_datetime(fff_m.index, format="%Y%m") # homogenous date format
    fff_m.index = fff_m.index.to_period("M") 
    return fff_m
    

In [6]:
get_fff().tail()

,Mkt-RF,SMB,HML,RF
2025-03,-0.0639,-0.0276,0.0290,0.0034
2025-04,-0.0084,-0.0059,-0.0340,0.0035
2025-05,0.0606,0.0070,-0.0288,0.0038
2025-06,0.0486,0.0083,-0.0160,0.0034
2025-07,0.0198,0.0027,-0.0126,0.0034


In [7]:
def capm(asset, start="2015-07", end="2025-07"):
    """
    Simple Linear Regression model with the market as sole factor, for one asset.

    Parameters
    ----------
    asset : pd.DataFrame
        raw data obtained with yf.download('Ticker', auto_adjust = False)
    start : pd.Period
        start date of data used.
    end: pd.Period
        end date of data used.
    """
    asset_m = monthly_returns(asset=asset).loc[start:end]
    fff_m = get_fff().loc[start:end]
    asset_excess = asset_m - fff_m.loc[:,['RF']].values
    mkt_excess = fff_m.loc[:,['Mkt-RF']]
    exp_var = mkt_excess.copy()
    exp_var["alpha"] = 1 # We add the constant for the linear regression
    return sm.OLS(asset_m, exp_var).fit()
    

In [8]:
lm = capm(small_cap)
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj Close   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     418.6
Date:                Wed, 08 Oct 2025   Prob (F-statistic):           8.85e-41
Time:                        13:32:20   Log-Likelihood:                 260.32
No. Observations:                 121   AIC:                            -516.6
Df Residuals:                     119   BIC:                            -511.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         1.1462      0.056     20.461      0.000       1.035       1.257
alpha         -0.0033      0.003     -1.234      0.220      -0.008       0.002
==============================================================================
Omnibus:                        9.686   Durbin-Watson:                   2.105
Prob(Omnibus):                  0.008   Jarque-Bera (JB):               11.659
Skew:                           0.480   Prob(JB):                      0.00294
Kurtosis:                       4.179   Cond. No.                         21.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
def fama_french_3f(asset, start="2015-07", end="2025-07"):
    """
    Multiple Linear Regression model with the 3 Factors (Market, SMB, HML), for one asset.

    Parameters
    ----------
    asset : pd.DataFrame
        raw data obtained with yf.download('Ticker', auto_adjust = False)
    start : pd.Period
        start date of data used.
    end: pd.Period
        end date of data used.
    """
    asset_m = monthly_returns(asset=asset).loc[start:end]
    fff_m = get_fff().loc[start:end]
    asset_excess = asset_m - fff_m.loc[:,['RF']].values
    mkt_excess = fff_m.loc[:,['Mkt-RF']]
    exp_var = mkt_excess.copy()
    exp_var["Value"] = fff_m.loc[:,["HML"]]
    exp_var["Size"] = fff_m.loc[:,["SMB"]]
    exp_var["alpha"] = 1 # We add the constant for the linear regression
    return sm.OLS(asset_m, exp_var).fit()
    

In [10]:
lm = fama_french_3f(small_cap)
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj Close   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     1705.
Date:                Wed, 08 Oct 2025   Prob (F-statistic):           2.41e-96
Time:                        13:32:20   Log-Likelihood:                 399.00
No. Observations:                 121   AIC:                            -790.0
Df Residuals:                     117   BIC:                            -778.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.9888      0.019     52.058      0.000       0.951       1.026
Value          0.3911      0.022     17.889      0.000       0.348       0.434
Size           0.7931      0.032     25.146      0.000       0.731       0.856
alpha          0.0007      0.001      0.860      0.391      -0.001       0.002
==============================================================================
Omnibus:                        0.129   Durbin-Watson:                   2.091
Prob(Omnibus):                  0.937   Jarque-Bera (JB):                0.008
Skew:                          -0.016   Prob(JB):                        0.996
Kurtosis:                       3.023   Cond. No.                         39.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
def fama_french_3f_multi(assets, start="2015-07", end="2025-07"):
    """
    Multiple Linear Regression model with the 3 Factors (Market, SMB, HML), for multiple assets.

    Parameters
    ----------
    assets : dict of pd.DataFrame
        dict of raw data obtained with yf.download('Ticker', auto_adjust = False)
    start : pd.Period
        start date of data used.
    end: pd.Period
        end date of data used.
    """
    # Results data-frame:
    subcols = ['value', 'p_value']
    columns = pd.MultiIndex.from_product([assets.keys(), subcols])
    results = pd.DataFrame(None, index=range(5), columns=columns)
    results.index = ['Mkt-RF', 'Value', 'Size', 'alpha', 'Rsquared']

    # Iterations
    for name, asset in assets.items():
        lm = fama_french_3f(asset, start=start, end=end)
        for i, param in enumerate(results.index[:-1]):
            results.loc[param, (name, 'value')] = lm.params[param]
            results.loc[param, (name, 'p_value')] = lm.pvalues[param]
        results.loc['Rsquared', (name, 'value')] = lm.rsquared
        
    return results

# Analysis and Results

In [12]:
etfs = {
    'Small_cap': small_cap,
    'Tech': tech,
    'Growth': growth,
    'Value': value
}

In [13]:
analysis = fama_french_3f_multi(etfs)

analysis.style.set_table_styles([
    {'selector': 'th', 'props': [('padding', '0 15px'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '0 15px')]}
]).format("{:.3f}")

## Comments

For each model, the R² is very high (close to 1), indicating that the model explains well the variance of returns.

**Factors beta interpretation:**

Now, we can interpret the factors beta for each ETF:

| Fund | Result |
|------------|------------|
| **iShares Core S&P Small-Cap ETF** | As expected, the SMB beta is positive and high, the ETF is exposed to small caps returns. The portfolio follows the market closely, as Market beta is close to 1. From the positive value factor beta in the regression, we interpret that small caps are mostly made of Value stock.|
| **iShares U.S. Technology ETF** | This ETF has a market beta over 1, meaning the market performances are amplified. This makes sense because the tech industry showed good performances during the last decade. Although the size beta p-value is 0.66 (results are not completely interpretable), we can note a negative size effect, explained by the portfolio large exposition to Mega Caps (Nvidia, Microsoft, Apple...)|
| **iShares S&P 500 Growth ETF** | The tech exposure of this portfolio made the results close to the Technology ETF and we can make the same interpretations. Moreover, the value factor is as anticipated negative. |
| **iShares S&P 500 Value ETF** | The Value ETF does not replicate the market as exactly as the other portfolios and is less sensitive to market variations. As expected there is a high exposition to the value factors. The size factor can't be interpreted and is close to 0. |


**Alpha interpretation:**

For the Small-Cap and the Value portfolios, alpha has a value of 0, even if the p-value is very high, we can interpret these alphas as noise.
The Technology ETF has a significant (relative to p-value) alpha of 0.6% highlighting a good management strategy. However, adding a tech sector factor could wash out this alpha.
For the Growth portfolio, the alpha is 0.2% and significant. We can make the same assumption as before of a hidden technology factor, since the 9 largest holding of this ETF (50% of the portfolio) are in the Communication, Information Technology and Consumer discretionary (Amazon, Tesla).

# References

- Kenneth R. French Data Library, **Darthmouth Tuck School of Business**
- **iShares** website (ETF descriptions and compositions), accessed on October 4, 2025
- Advanced Portfolio Construction and Analysis with Python, **EDHEC Business School** (Coursera MOOC)
- Is this time different? Perspective on the growth-vs-value debate, **Russel Investments** website, published on July 15, 2021